Set up environment

In [1]:
from collections import OrderedDict
import copy
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
sns.set_style("white")
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.anova import anova_lm
from sklearn.preprocessing import scale

Import custom modules

In [2]:
import sys
sys.path.append('../')
sys.path.append('../../utils/')

# Helper functions for eGLM 
from eGLM_helpers import phi, run_ucr_glm, run_ext_glm, make_stimtimes, sim_network_task_glm,  get_true_baseline, plot_sim_network_glm
import model

SyntaxError: invalid syntax (eGLM_helpers.py, line 39)

# Network structure

Goal of this notebook: Create the most minimal RNN that will be used in the more extended simulations to understand how activity propogates depending on the various parameters.

The network model describes change in activity as follows:

\begin{equation*}
\frac{dx_i}{dt}\tau_i = -x_i(t) + s\phi\big(x_i(t)\big) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(x_j(t)\big)\Bigg) + I_i(t)
\end{equation*}

We create network with 9 nodes with the following connectivity matrix. The first community (nodes 0-2) is a hub community and the other two are local communities.

In [ ]:
ncommunities = 3
innetwork_dsity = .6
outnetwork_dsity = .08
hubnetwork_dsity = .5
nodespercommunity = 3
plot_network = False

totalnodes = nodespercommunity*ncommunities

In [ ]:
# Construct structural matrix
S = model.generateStructuralNetwork(ncommunities=ncommunities,
                                    innetwork_dsity=innetwork_dsity,
                                    outnetwork_dsity=outnetwork_dsity,
                                    hubnetwork_dsity=hubnetwork_dsity,
                                    nodespercommunity=nodespercommunity,
                                    showplot=True)

plt.hlines(y=2.5, xmin = -0.5, xmax = 8.5)
plt.hlines(y=5.5, xmin = -0.5, xmax = 8.5)
plt.vlines(x=2.5, ymin = -0.5, ymax = 8.5)
plt.vlines(x=5.5, ymin = -0.5, ymax = 8.5)

Note in this matrix:

`W[..., x]` : column x of matrix denotes all outgoing connection weights from node x  
`W[x, ...]` : row x of matrix denotes all incoming connection weights to node x

In [ ]:
# Construct synaptic matrix
W = model.generateSynapticNetwork(S, showplot=False)

In [ ]:
Tmax = 1000
dt = 1
T = np.arange(0,Tmax,dt)

# Construct a community affiliation vector
Ci = np.repeat(np.arange(ncommunities),nodespercommunity) 
# Identify the regions associated with the hub network (hub network is by default the 0th network)
hub_ind = np.where(Ci==0)[0] 

stimsize = nodespercommunity

# This works because if there is a hub network the first nodes are the hub nodes
stim_nodes_td = np.arange(0, stimsize, dtype=int)
#stim_nodes_td

# Task structure

This network is stimulated topdown (i.e. only the hub nodes) with the following task:

In [ ]:
sa = 50
ea = 100
iv = 200
tasktiming, stimtimes = make_stimtimes(Tmax=1000, dt=1, stim_nodes= stim_nodes_td, stim_mag=0.5, sa = sa, ea = ea, iv = iv,
                          ncommunities = ncommunities, nodespercommunity = nodespercommunity)
plt.plot(tasktiming)

In [ ]:
taskdata, _ = model.networkModel(W, Tmax=Tmax,dt=dt,
                                     g=1, s=1, tau=1, I=stimtimes, 
                                     noise=None)

# Network activity time course

Timeseries/activity for each node in each community.  

Since only the hub network is stimulated by the task the amount of activity will depend on number (/strength) of incoming connections from hub network nodes. If a node does not have any incoming connections from the hub network its activity will not change from 0.

Connections from the hub community to the other communities can be checked on the columns going out of the hub nodes (in this case the first three columns).

In [ ]:
#Visualize what is going on on the network
#Three scatter plots with three curves for the timeseries/activity of each node in that community
taskdata_df = pd.DataFrame(taskdata)
taskdata_df['node_num'] = taskdata_df.index
taskdata_df['com_num'] = [0,0,0,1,1,1,2,2,2]
taskdata_df = taskdata_df.melt(id_vars = ['node_num', 'com_num'])
taskdata_df = taskdata_df.rename(columns={"variable": "time", "value": "activity"})

In [ ]:
plt.rcParams["figure.figsize"][0] = 20
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 3)

for cur_com in taskdata_df.com_num.unique():
    
    cur_vals = ["C"+str(x) for x in range(len(taskdata_df.query("com_num==@cur_com").node_num.unique()))]
    cur_col_lookup = dict(zip(taskdata_df.query("com_num==@cur_com").node_num.unique(), cur_vals))
    
    for cur_node in taskdata_df.query("com_num==@cur_com").node_num.unique():
        tmp = taskdata_df.query("com_num==@cur_com & node_num==@cur_node")
        a[cur_com].plot(tmp['time'], tmp['activity'], color = cur_col_lookup[cur_node], label = cur_node)
        a[cur_com].set_ylim([-0.2,2])
        a[cur_com].set_title("com_num = %s"%(str(cur_com)))
        a[cur_com].legend(loc="upper right")

Note that activity builds up and decreases in several time steps:

In [ ]:
plt.rcParams["figure.figsize"][0] = 14
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 2)

node = 0
task_on_t = np.where(np.diff(tasktiming) != 0)[0][0]
task_off_t = np.where(np.diff(tasktiming) != 0)[0][1]
range_start = 45
range_end = 70
plt_ranges = [range(range_start, range_end), range(range_start+50, range_end+50)]

a[0].plot(taskdata[node, plt_ranges[0]])
a[0].annotate("Task On", xy=(task_on_t - range_start-1, 0.35), xytext=(task_on_t - range_start-1, 0.35))
a[0].arrow(task_on_t - range_start, 0.3, 0, -0.2, head_width=0.5, head_length=0.1, fc='k', ec='k')
a[0].set_xticks([])
a[0].set_yticks([])
a[0].set_xlabel("Time")
a[0].set_ylabel("Activity")
a[0].set_ylim([-0.2,2])

a[1].plot(taskdata[node, plt_ranges[1]])
a[1].annotate("Task Off", xy=(task_off_t - (range_start+50)-1,  max(taskdata[node]) - .4), xytext=(task_off_t - (range_start+50)-1,  max(taskdata[node]) - .4))
a[1].arrow(task_off_t - (range_start+50), max(taskdata[node]) - .3, 0, 0.2, head_width=0.5, head_length=0.1, fc='k', ec='k')
a[1].set_xticks([])
a[1].set_yticks([])
a[1].set_xlabel("Time")
a[1].set_ylabel("")
a[1].set_ylim([-0.2,2])


# Comparison of design matrices

Back to the data generating process:

\begin{equation*}
\frac{dx_i}{dt}\tau_i = -x_i(t) + s\phi\big(x_i(t)\big) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(x_j(t)\big)\Bigg) + I_i(t)
\end{equation*}

\begin{equation*}
x_i(t+1) = \frac{dx_i}{dt}\tau_i + x_i(t)
\end{equation*}

So the GLM to invert should be:

\begin{equation*}
x_i(t+1) = s\phi\big(x_i(t)\big) + g\Bigg(\sum_{j\neq i}^{N} W_{ij}\phi\big(x_j(t)\big)\Bigg) + I_i(t)
\end{equation*}

The extended GLM contains additional regressors to account for the node's own activity (recurrence) and the network activity (functional connectivity)

If we were to run an experiment we would not know $I_i(t)$, i.e. how the task affects each node. Instead we would only know the task structure that we impose and check for the relationship between this time course and the time course of activity in each node (operationally this means using `tasktiming` instead of `stimtimes` for the task regressor).

In [ ]:
s = 1
g = 1
node = 0

#Drop the first time point
y = taskdata[node,1:]

#intercept
intcp = np.ones_like(y)

#Drop last time point
s_phi_x = s*phi(taskdata[node,:-1])

g_w_phi_x = np.delete(taskdata, node, axis=0)[:,:-1]
g_w_phi_x = np.apply_along_axis(phi, 0, g_w_phi_x)
cur_w = np.delete(W[node,:], node, axis=0)
cur_w = cur_w.reshape(-1,1)
g_w_phi_x = cur_w * g_w_phi_x
g_w_phi_x = np.sum(g_w_phi_x, axis=0)
g_w_phi_x = g*g_w_phi_x

# This is external; don't know how the task affects a node
# i_t = stimtimes[node,:-1]
i_t = tasktiming[:-1]

old_des_mat = np.column_stack((intcp, i_t))
new_des_mat = np.column_stack((intcp, s_phi_x, g_w_phi_x, i_t))

In [ ]:
plt.rcParams["figure.figsize"][0] = 10
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 3, gridspec_kw={'width_ratios': [1, 2, 4]})

a[0].set_title("Y", fontdict = {'fontsize':14})
a[1].set_title("cGLM design matrix", fontdict = {'fontsize':14})
a[2].set_title("eGLM design matrix", fontdict = {'fontsize':14})

sns.heatmap(y.reshape(-1,1), xticklabels=False, yticklabels=False, cbar=False, ax = a[0], vmin = 0 , vmax = 1)
sns.heatmap(old_des_mat, xticklabels=False, yticklabels=False, cbar=False, ax = a[1], vmin = 0 , vmax = 1)
sns.heatmap(new_des_mat, xticklabels=False, yticklabels=False, cbar=True, ax = a[2], vmin = 0 , vmax = 1)

plt.subplots_adjust(wspace=0.1)

In [ ]:
mod_df = pd.DataFrame(data = {"y": y, "s_phi_x":s_phi_x, "g_w_phi_x":g_w_phi_x, "i_t":i_t})
old_mod = smf.ols(formula = 'y ~ i_t', data = mod_df).fit()
mid_mod = smf.ols(formula = 'y ~ s_phi_x + g_w_phi_x', data = mod_df).fit()
new_mod = smf.ols(formula = 'y ~ s_phi_x + g_w_phi_x + i_t', data = mod_df).fit()

In [ ]:
print("cGLM task parameter estimate: %s"%(str(round(old_mod.params['i_t'],3))))
print("eGLM task parameter estimate: %s"%(str(round(new_mod.params['i_t'],3))))

Note: $s$ and $g$ are network properties that you wouldn't know about in emprical task data. But these might be estimated from resting state data and then plugged in for task analyses.  

# Baseline calculation

To assess the "improvement" of the model we would like to compare the task involvement estimates to a "true" baseline since the data is synthetic. 

Standard model comparison of the reduction in sum of squared residuals as a function of the degrees of freedom isn't too informative with respect to our desired goal because a model with more parameters will always account for more of the variance in the data. We are interested in whether the variance is attributed to the correct regressors.  

Change in average activity when task is on compared to when task is off does not provide the "true" change in signal due to task becaus it is "contaminated" by network activity.  

Baseline should capture how much of the change in $y$ is due to $I(t)$ separate from the effect of recurrence and functional connectivity. In other words it is the regression weight of $I(t)$ in the extended model for **noiseless data**
This way, baseline does **NOT** depend on noise but it **DOES** depend on the node's connectivity (so it wouldn't be the same for all nodes, even all stimulated nodes).

## Stimulated node

The plot below shows the time course of one stimulated node's activity following a task on and task off period (dashed line). Additionally it also depicts the time course of each of the regressors and the predicted node activity time series from models with increasing complexity.

If we were to describe the effect of task on the node's time course using the raw time course of the task activity (blue line). This would overestimate the task involvement (the classic GLM case) as it must be multiplied with a large factor to approximate the dashed line. Furthermore as seen in the model predicted values (red line) there are systematic over and under estimations using just this regressor.

If we instead think of the data as a linear combination (brown line) of the task activity (blue), recurrent activity (yellow) and network activity (green) then we get a better approximation of the data. The task regressor coefficient in this case is our baseline.

In [ ]:
plt.rcParams["figure.figsize"][0] = 18
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 2)

plt_ranges = [range(45,60), range(90,120)]
titles = ['Task On', "Task Off"]

for i, plt_range in enumerate(plt_ranges):
    a[i].plot(i_t[plt_range], label = "i_t")
    a[i].plot(s_phi_x[plt_range], label = "s_phi_x")
    a[i].plot(g_w_phi_x[plt_range], label = "g_w_phi_x")
    a[i].plot(old_mod.predict()[plt_range], label="y ~ i_t")
    a[i].plot(mid_mod.predict()[plt_range], label='y ~ s_phi_x + g_w_phi_x')
    a[i].plot(new_mod.predict()[plt_range], label='y ~ s_phi_x + g_w_phi_x + i_t', linewidth=5)
    a[i].plot(y[plt_range], label = "y", linewidth=5, linestyle='--')
    a[i].legend()
    a[i].set_title(titles[i], fontdict = {'fontsize':14})
    a[i].set_xticks([])
    a[i].set_yticks([])
    a[i].set_ylim([-.2,2.5])
    
plt.subplots_adjust(wspace=0.1)
plt.suptitle("Stimulated node", size=16)

In [ ]:
mod_df['task_on'] = np.where(mod_df['i_t']==0,"off","on")
sns.pairplot(mod_df, hue='task_on', vars=mod_df.columns[:-1], diag_kind="hist")

## Non-stimulated node

**Why is the baseline not 0?**

In [ ]:
s = 1
g = 1
node = 6

#Drop the first time point
y = taskdata[node,1:]

#intercept
intcp = np.ones_like(y)

#Drop last time point
s_phi_x = s*phi(taskdata[node,:-1])

g_w_phi_x = np.delete(taskdata, node, axis=0)[:,:-1]
g_w_phi_x = np.apply_along_axis(phi, 0, g_w_phi_x)
cur_w = np.delete(W[node,:], node, axis=0)
cur_w = cur_w.reshape(-1,1)
g_w_phi_x = cur_w * g_w_phi_x
g_w_phi_x = np.sum(g_w_phi_x, axis=0)
g_w_phi_x = g*g_w_phi_x

# This is external; don't know how the task affects a node
# i_t = stimtimes[node,:-1]
i_t = tasktiming[:-1]

old_des_mat = np.column_stack((intcp, i_t))
new_des_mat = np.column_stack((intcp, s_phi_x, g_w_phi_x, i_t))

mod_df = pd.DataFrame(data = {"y": y, "s_phi_x":s_phi_x, "g_w_phi_x":g_w_phi_x, "i_t":i_t})
old_mod = smf.ols(formula = 'y ~ i_t', data = mod_df).fit()
mid_mod = smf.ols(formula = 'y ~ s_phi_x + g_w_phi_x', data = mod_df).fit()
new_mod = smf.ols(formula = 'y ~ s_phi_x + g_w_phi_x + i_t', data = mod_df).fit()

print("cGLM task parameter estimate: %s"%(str(round(old_mod.params['i_t'],3))))
print("eGLM task parameter estimate: %s"%(str(round(new_mod.params['i_t'],3))))

plt.rcParams["figure.figsize"][0] = 18
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 2)

plt_ranges = [range(45,60), range(90,120)]
titles = ['Task On', "Task Off"]

for i, plt_range in enumerate(plt_ranges):
    a[i].plot(i_t[plt_range], label = "i_t")
    a[i].plot(s_phi_x[plt_range], label = "s_phi_x")
    a[i].plot(g_w_phi_x[plt_range], label = "g_w_phi_x")
    a[i].plot(old_mod.predict()[plt_range], label="y ~ i_t")
    a[i].plot(mid_mod.predict()[plt_range], label='y ~ s_phi_x + g_w_phi_x')
    a[i].plot(new_mod.predict()[plt_range], label='y ~ s_phi_x + g_w_phi_x + i_t', linewidth=5)
    a[i].plot(y[plt_range], label = "y", linewidth=5, linestyle='--')
    a[i].legend()
    a[i].set_title(titles[i], fontdict = {'fontsize':14})
    a[i].set_xticks([])
    a[i].set_yticks([])
    a[i].set_ylim([-.2,2.5])
    
plt.subplots_adjust(wspace=0.1)
plt.suptitle("Non-stimulated node", size=16)

Correlation between regressors

In [ ]:
mod_df['task_on'] = np.where(mod_df['i_t']==0,"off","on")
sns.pairplot(mod_df, hue='task_on', vars=mod_df.columns[:-1], diag_kind="hist")

# Task parameter correction

In [ ]:
dt = 1
tau = 1
g = 1
s = 1 
stim_mag = .5
sa = 50
ea = 100
iv = 200

base_sim = sim_network_task_glm(ncommunities = ncommunities, 
                         nodespercommunity = nodespercommunity, 
                         dt = dt, tau = tau, g = g, s = s, 
                         Tmax = Tmax, 
                         stimsize = nodespercommunity, 
                         stim_mag = stim_mag,
                         W = W,
                         taskdata = taskdata,
                         tasktiming = tasktiming,
                         sa = sa,
                         ea = ea,
                         iv = iv,
                         standardize=False)

In [ ]:
plot_sim_network_glm(base_sim, nnods = 3)